### Homework 07: Concurrency

## Due Date: Apr 12, 2024, 6:00pm

#### Firstname Lastname: Jiasheng Ni

#### E-mail: jn2294@nyu.edu

#### Enter your solutions and submit this notebook


---

**Problem 1** **(60 Points)**

Let us consider the Gamma function, or the Euler integral of the second kind: 

$$\Gamma(x) = \int_{0} ^ \infty t ^{x - 1} e^{-t} dt, $$

and in this HW we consider real $x > 0$.

(Here is more on the Gamma function https://en.wikipedia.org/wiki/Gamma_function .
It is not needed for this HW assignment.) 

**1.1 (Points 15)**: 

Write a function (in the cell below) that sequentially calculates the given Gamma integral.


In [6]:
import numpy as np
import time

def func(t, x):
    return np.power(t, x-1) * np.exp(-1 * t)


def calculate_gamma(x, bound_1, bound_2, number_of_steps):
    t_segs = np.linspace(bound_1, bound_2, number_of_steps + 1)
    step_size = (bound_2 - bound_1) / (number_of_steps)

    integral_res = 0
    for t in t_segs[:-1]:
        integral_res += func(t, x) * step_size

    return integral_res

**1.2 (Points 5)** 

Evaluate, $\Gamma(6)$ by using `calculate_gamma(x, bound_1, bound_2, number_of_steps)` and the error of this computation.


As arguments, use `x=6, bound_1=0, bound_2=1000, number_of_steps=10_000_000`. We know that $\Gamma(x) = x!$, so $\Gamma(6) = 5! = 120$. 


In [7]:
start = time.time()
result = calculate_gamma(6, 0, 1000, 10000000)
error = 120 - result
print(f"Result: {result}, Time Lapsed: {time.time() - start}, error is {error}")

Result: 119.99999999994274, Time Lapsed: 12.738117456436157, error is 5.725553364754887e-11


---

Write two functions to calculate $\Gamma(x)$ by using:



**1.3.1 (Points 15)**
**threading** with N=4 threads; 

**1.3.2 (Points 15)**
**multiprocessing** with N=4 processes. 


**1.3.3 (Points 10)** 
Compare the times of the three versions and write a short explanation of what you are observing.

How does the answer change when N=8 and why?



In [4]:
import time
import threading
import numpy as np
from queue import Queue
from threading import Thread
from threading import Lock



def multithreading_calculate_gamma(x, bound_1, bound_2, number_of_steps, num_of_threads):
    # Init
    steps = np.linspace(bound_1, bound_2, number_of_steps + 1)
    step_size = (bound_2 - bound_1) / number_of_steps
    chunk_size = number_of_steps // (num_of_threads - 1)
    # Consider the floor division

    chunks = [(i, i + chunk_size) for i in range(0, number_of_steps - chunk_size, chunk_size)] + [(chunk_size * (num_of_threads - 1), number_of_steps)]
    q = Queue()
    lock = Lock()

    # Accumulator
    results = 0

    # Worker Code for threads
    def sum_chunk(q, x):
        nonlocal results
        nonlocal steps
        nonlocal step_size
        local_result = 0
        while True:
            bound_1, bound_2 = q.get()
            for i in range(bound_1, bound_2):
                t = steps[i]
                local_result += func(t, x) * step_size
            with lock:  # force synchronization
                results = results + local_result
            q.task_done()

    for i in range(num_of_threads):
        worker = Thread(target=sum_chunk, args=(q, x))
        worker.setDaemon(True) # this stop the threads when the program quits
        worker.start()         # start the threads

    for chunk in chunks:
        q.put(chunk)

    q.join()

    return results

start = time.time()
result_4_thread = multithreading_calculate_gamma(6, 0, 1000, 10000000, 4)
error_4_thread = 120 - result_4_thread
print(f"Result: {result_4_thread}, Time Lapsed: {time.time() - start}, error is {error_4_thread}")

start = time.time()
result_8_thread = multithreading_calculate_gamma(6, 0, 1000, 10000000, 8)
error_8_thread = 120 - result_8_thread
print(f"Result: {result_8_thread}, Time Lapsed: {time.time() - start}, error is {error_8_thread}")

C:\Users\alexm\AppData\Local\Temp\ipykernel_63200\3195937392.py:41: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  worker.setDaemon(True) # this stop the threads when the program quits


Result: 119.99999999994274, Time Lapsed: 13.362726926803589, error is 5.725553364754887e-11
Result: 119.99999999994274, Time Lapsed: 12.574717283248901, error is 5.725553364754887e-11


In [10]:
!python multiprocessing_calculate_gamma.py -n 4

10000000
Result: 119.9999999999999, Time Lapsed: 17.162447452545166, error is 9.947598300641403e-14


In [9]:
!python multiprocessing_calculate_gamma.py -n 8

Result: 119.9999999999999, Time Lapsed: 16.590646982192993, error is 9.947598300641403e-14


Problem 1 Explanations


---

**Problem 2 (40 points)**

__Website uptime__ is the time that a website or web service is available to the users over a given period.

The task is to build an application that checks the uptime of websites. 

- The application should go over a list of website URLs and checks if those websites are up.
- Instead of performing a classic HTTP GET request, it performs a HEAD request so that it does not affect traffic significantly.
- If the HTTP status is in the danger ranges (400+, 500+), a message is casted. 

Here are some useful functions:

In [1]:
#### _website uptimer_ ####

import time
import logging
import requests
 
class WebsiteDownException(Exception):
    pass
 
def ping_website(address, timeout=20):
    """
    Check if a website is down. A website is considered down 
    if either the status_code >= 400 or if the timeout expires
     
    Throw a WebsiteDownException if any of the website down conditions are met
    """
    try:
        response = requests.head(address, timeout=timeout)
        if response.status_code >= 400:
            logging.warning("Website %s returned status_code=%s" % (address, response.status_code))
            raise WebsiteDownException()
    except requests.exceptions.RequestException:
        logging.warning("Timeout expired for website %s" % address)
        raise WebsiteDownException()
         
def check_website(address):
    """
    Utility function: check if a website is down, if so, notify the user
    """
    try:
        ping_website(address)
    except WebsiteDownException:
        print('The websie ' + address + ' is down')

---

You need a website list to try our system out. Create your own list or use the following one. 

---

In [2]:
WEBSITE_LIST = [
    'http://amazon.co.uk',
    'http://amazon.com',
    'http://facebook.com',
    'http://google.com',
    'http://google.fr',
    'http://google.es',
    'http://google.co.uk',
    'http://gmail.com',
    'http://stackoverflow.com',
    'http://github.com',
    'http://heroku.com',
    'http://really-cool-available-domain.com',
    'http://djangoproject.com',
    'http://rubyonrails.org',
    'http://basecamp.com',
    'http://trello.com',
    'http://shopify.com',
    'http://another-really-interesting-domain.co',
    'http://airbnb.com',
    'http://instagram.com',
    'http://snapchat.com',
    'http://youtube.com',
    'http://baidu.com',
    'http://yahoo.com',
    'http://live.com',
    'http://linkedin.com',
    'http://netflix.com',
    'http://wordpress.com',
    'http://bing.com',
]

---

A serial version of the _website uptimer_ can be written as: 

---


In [3]:
import time
 
start_time = time.time()
 
for address in WEBSITE_LIST:
    check_website(address)
         
end_time = time.time()        
 
print("Time for Serial: %ssecs" % (end_time - start_time))

The websie http://really-cool-available-domain.com is down


The websie http://another-really-interesting-domain.co is down
Time for Serial: 2.6749846935272217secs


You should build two versions of the **website uptimer**, by using:

**2.1 (Points 15)**
**threading** with N=4 threads; 

**2.2 (Points 15)**
**multiprocessing** with N=4 processes. 


**2.3 (Points 10)** 

Compare the times of the three versions and write a short explanation of what you are observing.

How does the answer change when N=8 and why?


In [3]:
import time
import threading
import numpy as np
from queue import Queue
from threading import Thread
from threading import Lock

def multi_threading_ping(websites, num_of_threads):

    num_of_websites = len(websites)
    chunk_size = num_of_websites // (num_of_threads - 1)
    chunk_indices = [(i, i + chunk_size) for i in range(0, len(websites) - chunk_size, chunk_size)] + [(chunk_size *(num_of_threads - 1), len(websites))]

    chunk_websites = [[websites[j] for j in range(chunk_index[0], chunk_index[1])] for chunk_index in chunk_indices]

    q = Queue()

    def ping_chunk(q):
        while True:
            website_addrs = q.get()
            for website_addr in website_addrs:
                check_website(website_addr)
            q.task_done()

    for i in range(num_of_threads):
        worker = Thread(target=ping_chunk, args=(q,))
        worker.setDaemon(True) # this stop the threads when the program quits
        worker.start()         # start the threads

    for chunk in chunk_websites:
        q.put(chunk)


    q.join()

In [5]:
num_of_threads = 4
start = time.time()
multi_threading_ping(WEBSITE_LIST, num_of_threads)
print(f"Num of Threads: {num_of_threads}, Time Lapse: {time.time() - start}")

num_of_threads = 8
start = time.time()
multi_threading_ping(WEBSITE_LIST, num_of_threads)
print(f"Num of Threads: {num_of_threads}, Time Lapse: {time.time() - start}")

C:\Users\alexm\AppData\Local\Temp\ipykernel_95784\2758633146.py:27: DeprecationWarning: setDaemon() is deprecated, set the daemon attribute instead
  worker.setDaemon(True) # this stop the threads when the program quits


The websie http://really-cool-available-domain.com is down


The websie http://another-really-interesting-domain.co is down


Num of Threads: 4, Time Lapse: 1.0804872512817383
The websie http://another-really-interesting-domain.co is down
The websie http://really-cool-available-domain.com is down
Num of Threads: 8, Time Lapse: 0.8260636329650879


In [16]:
!python multiprocessing_ping.py -n 4

Time Lapsed: 1.3548572063446045
The website http://really-cool-available-domain.com is down
The website http://another-really-interesting-domain.co is down


In [17]:
!python multiprocessing_ping.py -n 8

Time Lapsed: 1.066051721572876
The website http://another-really-interesting-domain.co is down
The website http://really-cool-available-domain.com is down


### Problem 2 Explanations
We see that for this task, compared with N = 4, when N = 8, we ping the websites faster.